In [2]:
import polars as pl
import numpy as np
import dsds.metrics as me
import dsds.prescreen as ps
import dsds.sample as sa
import dsds.fs as fs
import dsds.transform as t
import numpy as np 

In [ ]:
df = pl.DataFrame({
    "time": ["2021-01-01", "2021-01-03", "2021-02-01","2021-02-11","2021-03-01","2021-03-02"],
    "a1": [None, 1,2,3,4, None,],
    "a2": [1,2, None,3,4, None,],
})
df = df.with_columns(pl.col("time").str.to_date())
ps.over_time_report(df, cols=["a1", "a2"], time_col="time", metrics=["null", "invalid", "min", "max"])

In [ ]:
int(3.5)

In [ ]:
df = pl.scan_csv("../data/advertising.csv")
df.select(['id',
 'Daily Time Spent on Site',
 'Age',
 'Area Income',
 'Daily Internet Usage',
 'Ad Topic Line',
 'City',
 'Male',]).drop(['id',
 'Daily Time Spent on Site']).show_graph()

In [ ]:
%%timeit
df.select(
    pl.max_horizontal(pl.col("a"), pl.lit(0.1))
)

In [ ]:
%%timeit
df.select(
    pl.col("a").clip_min(0.1)
)

In [ ]:
pl.show_versions()

In [ ]:
t.scale(df, cols=["a","b"], strategy="robust")

In [ ]:
import dsds.transform as t
df = pl.DataFrame({
    "a": ['{ "id": 1, "name": "a", "course": {"0":"abc"}}', '{ "id": 2, "name": "b", "course": {"0":"efg"}}']
})

In [ ]:
df = pl.read_csv("../data/advertising.csv")
df.head()

In [ ]:
predicted = np.random.random(size=(5000,3))
actual = np.round(np.random.random(size=(5000,3))).astype(np.int8)

In [ ]:
df = pl.DataFrame({
    "a": predicted[:, 1],
    "b": actual[:, 1]
})
df 

In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(actual, predicted, average="weighted"))
print(me.roc_auc(actual, predicted, strategy="balanced"))

In [ ]:
%%timeit
roc_auc_score(actual, predicted)

In [ ]:
%%timeit
me.roc_auc(actual, predicted, strategy="none")

In [ ]:
%%timeit
roc_auc_score(actual, predicted)

In [ ]:
%%timeit
me.roc_auc(actual, predicted)

In [ ]:
import dsds.prescreen as ps
print(ps.corr_table(df, cols=["Age", "Daily Internet Usage"], corr_with=["Clicked on Ad", "Age Band"]))

In [ ]:
df2 = (
    df.lazy().select(["Area Income", "Ad Topic Line", "City", "Clicked on Ad"])
    .drop(["Ad Topic Line"])
).collect()

In [ ]:
df2.show_graph()

In [ ]:
from scipy.fft import fft
a = np.array([1,2,3,1,2,3,1,2,3])
fft(a)

In [ ]:
df = pl.read_csv("../data/train.csv").with_columns(
    pl.lit(1).alias("feature_1"),
    pl.lit(2).alias("feature_2")
)

In [ ]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures

In [ ]:
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

In [ ]:
timeseries

In [ ]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

In [ ]:
extracted_features